In [21]:
# importing the necessary packages
import numpy as np
import pandas as pd
import os
import pickle
import requests
import shutil
from tqdm import tqdm


# setting the main file path to where all files are
os.chdir("C:\\Users\\SJNic\\Documents\\University\\Master\\Thesis\\Thesis Code")

# setting the directory for where to save the document files
documents_dir = "PDFs/Documents/"

In [43]:
# read the documentsInformation file
with open("NewProcessingData/documentsInformation.txt", "rb") as fp:   
    documentsInformation = pickle.load(fp)

# Download every doc in the list via the url

In [48]:
unique_docs_dir = "PDFs/UniqueDocuments"

downloaded_docs = [d[0:-4] for d in os.listdir(unique_docs_dir)]

i=0
for key in tqdm(documentsInformation):
    #if i > 10:
    #    break
        
    doc = documentsInformation[key]
    
    # get the unique version of this doc
    uniqueDoc = documentsInformation[doc["unique id"]]
    uniqueDocID = uniqueDoc["document id"]
    #print(uniqueDocID)
    
    try: 
        #s = "/" + uniqueDocID
        # check whether this doc has already been downloaded
        if uniqueDocID not in downloaded_docs:
            i+=1
    except:
        print(uniqueDocID)
i

100%|██████████| 52137/52137 [00:05<00:00, 9371.65it/s] 


41465

In [54]:
for key in tqdm(documentsInformation):
    doc = documentsInformation[key]
    if type(doc["unique id"]) != str:
        
        print(type(doc["unique id"]))
    if str(type(doc["unique id"])) == "<class 'dict'>":
        print(doc["unique id"])

100%|██████████| 52137/52137 [00:00<00:00, 743288.42it/s]


In [55]:
unique_docs_dir = "PDFs/UniqueDocuments"

downloaded_docs = [d[0:-4] for d in os.listdir(unique_docs_dir)]

for key in tqdm(documentsInformation):
    doc = documentsInformation[key]
    
    # get the unique version of this doc
    uniqueDoc = documentsInformation[doc["unique id"]]
    uniqueDocID = uniqueDoc["document id"]
    #print(uniqueDocID)
    
    # somtimes the reference is 
    if str(type(uniqueDocID))  == "<class 'dict'>":
        uniqueDocID = uniqueDocID["id"]
    
    # check whether this doc has already been downloaded
    if uniqueDocID not in downloaded_docs:
        downloaded_docs.append(uniqueDoc)
        
        # get the download url
        url = uniqueDoc["publicdownloadurl"]
        
        # download the doc and save it
        r = requests.get(url)
        open(unique_docs_dir + "/" + uniqueDocID + ".pdf", 'wb').write(r.content)
        
    #break 

100%|██████████| 52137/52137 [4:48:24<00:00,  3.01it/s]   


In [ ]:

for docId in documentsInformation.keys():
    doc = documentsInformation[docId]
    url = doc["publicdownloadurl"]

    r = requests.get(url)
    open(documents_dir + docId + ".pdf", 'wb').write(r.content)


# checking if we have all unique IDS

In [10]:
with open("NewProcessingData/unique_IDs.txt", "rb") as fp:
    unique_IDs = pickle.load(fp)

In [11]:
downloaded_files = os.listdir(documents_dir)

not_downloaded_files = []

for doc in unique_IDs:
    # if this document is not in downloaded files then add it to the list of documents to download
    if doc not in downloaded_files:
        not_downloaded_files.append(doc)

In [12]:
len(not_downloaded_files)

48935

In [8]:
len(unique_IDs)

49145

# Duplicate detection

In [35]:
# this method only counts the number of duplicates based on a matching title
filenames = []
count = 0
for key in documentsInformation:
    duplicate = False
    doc = documentsInformation[key]
    filename = doc["filename"]
    # check if this filename has already been found
    for i in range(len(filenames)):
        if filename == filenames[i]:
            count += 1
            duplicate = True
            break
    if not duplicate:
        filenames.append(filename)
print("# of duplicates = ", count)

# of duplicates =  5525


In [37]:
# Now trying the same method but for the same file sizes
fileSizes = []
count = 0
for key in documentsInformation:
    duplicate = False
    doc = documentsInformation[key]
    fileSize = doc["filesize"]
    # check if this filename has already been found
    for i in range(len(fileSizes)):
        if fileSize == fileSizes[i]:
            count += 1
            duplicate = True
            break
    if not duplicate:
        fileSizes.append(fileSize)
print("# of duplicate file sizes = ", count)

# of duplicate file sizes =  5533


# Move duplicate pdfs to a seperate folder

In [7]:
with open("nonDuplicateDocumentsInformation.txt", "rb") as fp:   
    nonDuplicateDocumentsInformation = pickle.load(fp)

In [14]:
duplicateIDS = []
setValues = set(nonDuplicateIDS.values())

# first retrieve all ids of duplicate documents
for key in documentsInformation:
    if key not in setValues:
        duplicateIDS.append(key)
len(duplicateIDS)

5525

In [8]:
# now moving the duplicate documents
# folder to move the duplicate files to
duplicate_documents_dir = "Data/DuplicateDocuments/"
for id in duplicateIDS:
    os.rename(documents_dir + id + ".pdf", duplicate_documents_dir + id + ".pdf")

In [18]:
print(len(nonDuplicateDocumentsInformation))
print(len(duplicateIDS))
print(len(documentsInformation))

23704
5525
29229


# Reduce Data dimensionality

  Created wheel for pdfplumber: filename=pdfplumber-0.5.28-py3-none-any.whl size=32219 sha256=9cee7d80246c906e584a19ef02432564c9b8119a97ab018ca324bd1de7c0c2b7
  Stored in directory: c:\users\sjnic\appdata\local\pip\cache\wheels\36\61\6d\5fdf7f85a9598d42f094b4099be9a3dd9a887b25ca9b5a1bf4
Successfully built pdfplumber


In [3]:
import pdfplumber
from multiprocessing.dummy import Pool as ThreadPool


texts_dir = "Data/DocumentTexts/"
# setting the directory for where to save the document files
documents_dir = "Data/Documents/"

with open("nonDuplicateDocumentsInformation.txt", "rb") as fp:   
    nonDuplicateDocumentsInformation = pickle.load(fp)

In [3]:
key = "72cc4b22-0de8-4a83-ae9c-b4961b74050e"
test = nonDuplicateDocumentsInformation[key]
test

{'document id': '72cc4b22-0de8-4a83-ae9c-b4961b74050e',
 'displayname': 'Bijlage 1 Verzoek van de fracties van VVD, CDA en D66 tot het mogen houden van een debat over het feit dat het College de NOUW 3 niet heeft opgenomen in het Actieplan Luchtkwaliteit',
 'filename': 'Bijlage 1 Verzoek van de fracties van VVD, CDA en D66 tot het mogen houden van een debat over het feit dat het College de NOUW 3 niet heeft opgenomen in het Actieplan Luchtkwaliteit.pdf',
 'filesize': '131576',
 'publicdownloadurl': 'https://api1.ibabs.eu/publicdownload.aspx?site=Utrecht&id=72cc4b22-0de8-4a83-ae9c-b4961b74050e'}

In [4]:
def save_text_from_pdf(key):
    file_loc = documents_dir + key + ".pdf"
    text = ""
    with pdfplumber.open(file_loc) as pdf:
        
        for page in pdf.pages:
            try:
                text = text + page.extract_text()
            except:
                continue
                

    with open(texts_dir + key + ".txt", "wb") as fp:
            pickle.dump(text, fp)  

In [5]:
for key in nonDuplicateDocumentsInformation.keys():
    save_text_from_pdf(key)

PSEOF: Unexpected EOF

In [5]:
save_text_from_pdf(key)

In [ ]:
# making use of multithreading to hopefully increase the speed of text extraction
pool = ThreadPool(8)

pool.map(save_text_from_pdf, nonDuplicateDocumentsInformation.keys())

# Close the pool and wait for the work to finish
pool.close()
pool.join()

#save_text_from_pdf("72cc4b22-0de8-4a83-ae9c-b4961b74050e")